## Importing libraries

In [49]:
import pandas as pd 
import cbsodata
import json
import re
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

## Loading the data

In [ ]:
# Define the excluded table identifiers
excluded_identifiers = ['85663NED'] # Only has quarterly data since 2020 

# Load the data from the JSON file
with open('data/table_selection.json', 'r', encoding='utf-8') as json_file:
    table_data = json.load(json_file)

# Create a dictionary to store the identifiers and their frequencies
identifier_freq_dict = {}

# Loop through the data and extract the identifier and frequency for each entry
for entry in table_data:
    identifier = entry.get('Identifier', 'N/A')
    frequency = entry.get('Frequency', 'N/A')
    
    # Skip identifiers that are in the excluded_identifiers list
    if identifier in excluded_identifiers:
        continue
    
    # Add the identifier and frequency to the dictionary
    identifier_freq_dict[identifier] = frequency

# Print the dictionary of identifiers and their frequencies
print("Dictionary of Identifiers and Frequencies:")
for identifier, frequency in identifier_freq_dict.items():
    print(f"Identifier: {identifier}, Frequency: {frequency}")


In [ ]:
def fetch_and_save_tables(identifiers, output_folder='data'):
    """
    Fetches data for each table identifier using `cbsodata.get_data()` and saves it to a CSV file,
    skipping files that already exist.
    
    Parameters:
        identifiers (list): List of table identifiers to fetch data for.
        output_folder (str): The folder where the CSV files will be saved (default is 'data').
    
    Returns:
        None
    """
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for identifier in identifiers:
        # Define the output CSV file path
        output_file = f'{output_folder}/{identifier}.csv'
        
        # Check if the file already exists
        if os.path.exists(output_file):
            print(f"Skipping {output_file} because it already exists.")
            continue
        
        # Fetch data for the current table identifier
        print(f"Fetching data for table identifier: {identifier}")
        data = pd.DataFrame(cbsodata.get_data(identifier))
        
        # Save the DataFrame to CSV
        data.to_csv(output_file, index=False)
        
        # Print success message
        print(f"Data for {identifier} saved to {output_file}")

# Example usage
fetch_and_save_tables(identifier_freq_dict.keys())

In [52]:
def load_csv_files(file_paths, identifier_dict, excluded_identifiers):
    """
    Load multiple CSV files into DataFrames, prefix column names with the file name, 
    and convert monthly tables to quarterly, excluding specific identifiers.
    
    Parameters:
        file_paths (list): List of file paths to CSV files.
        identifier_dict (dict): Dictionary of identifiers and their frequencies.
        excluded_identifiers (list): List of identifiers to exclude.
    
    Returns:
        list: List of DataFrames with prefixed column names.
    """
    dataframes = []
    
    for file_path in file_paths:
        file_name = file_path.split('/')[-1].split('.')[0]
        
        # Skip files with identifiers in the excluded list
        if file_name in excluded_identifiers:
            print(f"Skipping excluded file: {file_name}")
            continue
        
        frequency = identifier_dict.get(file_name, 'Kwartaal')  # Default to 'Kwartaal' if not specified

        # Load CSV file
        df = pd.read_csv(file_path)
        
        # Prefix column names with filename
        df = df.add_prefix(f"{file_name}_")
        
        # Rename specific columns to prevent prefixing
        for col in ['Perioden', 'BedrijfstakkenBranchesSBI2008', 'ID', 'Jaar', 'Kwartaal']:
            if f"{file_name}_{col}" in df.columns:
                df = df.rename(columns={f"{file_name}_{col}": col})
                
        # If the frequency is monthly, convert it to quarterly
        if frequency == 'Maandelijks':
            print(f"Converting monthly data to quarterly for file: {file_name}")
            df = monthly_to_quarterly(df)

        # Append to list
        dataframes.append(df)
    
    return dataframes

# Function to process the 'Perioden' column
def process_period_column(df, perioden_column='Perioden', year_column='Year', quarter_column='Quarter'):
    """
    Process the 'Perioden' column to extract the latest year and quarter.
    
    Parameters:
        df (DataFrame): DataFrame with 'Perioden' column.
        perioden_column (str): Name of the period column.
        year_column (str): Name of the column for extracted year.
        quarter_column (str): Name of the column for extracted quarter.
    
    Returns:
        DataFrame: DataFrame with new 'Year' and 'Quarter' columns.
    """
    # Extract latest year and quarter from 'Perioden' column
    df[[year_column, quarter_column]] = df[perioden_column].apply(lambda x: pd.Series(extract_latest_year_and_quarter(x)))
    return df

# Helper function to extract latest year and quarter
def extract_latest_year_and_quarter(period_text):
    """
    Extracts the latest year and quarter from a period that may contain a date range (e.g., "2023 april - 2024 maart").
    
    Parameters:
        period_text (str): The period string to process.
    
    Returns:
        tuple: A tuple containing the latest year and quarter.
    """
    # Regular expression to capture year and month/quarter references
    match = re.findall(r'(\d{4})\s*(\w+)', period_text)
    
    if match:
        # Take the last match in the range (latest part of the period)
        latest_year, latest_period_part = match[-1]
        latest_year = int(latest_year)
        
        # Determine the corresponding quarter for the latest period part (month or quarter)
        latest_quarter = extract_quarter(latest_period_part)
        
        return latest_year, latest_quarter
    else:
        # Return None if no valid year/quarter is found
        return None, None

# Helper function to determine the quarter based on months or quarter phrases like '3e kwartaal'
def extract_quarter(period_part):
    """
    Extracts the quarter from the period part (e.g., based on months or quarter keywords).
    
    Parameters:
        period_part (str): The part of the period string after the year.
    
    Returns:
        int: The corresponding quarter (1 to 4), or None if no valid quarter is found.
    """
    # Handle month-based quarters
    if any(month in period_part for month in ['januari', 'februari', 'maart']):
        return 1
    elif any(month in period_part for month in ['april', 'mei', 'juni']):
        return 2
    elif any(month in period_part for month in ['juli', 'augustus', 'september']):
        return 3
    elif any(month in period_part for month in ['oktober', 'november', 'december']):
        return 4
    
    # Handle phrases like '1e kwartaal', '2e kwartaal', etc.
    if '1e' in period_part:
        return 1
    elif '2e' in period_part:
        return 2
    elif '3e' in period_part:
        return 3
    elif '4e' in period_part:
        return 4
    
    # Return None if no valid quarter is found
    return None

def monthly_to_quarterly(df, period_column='Perioden', value_columns=None):
    """
    Convert monthly data to quarterly by summing up the data for each quarter.
    
    Parameters:
        df (DataFrame): The DataFrame with monthly data.
        period_column (str): Name of the column containing period information.
        value_columns (list): List of columns to sum (numeric columns).
    
    Returns:
        DataFrame: DataFrame with quarterly data.
    """
    if value_columns is None:
        value_columns = df.select_dtypes(include=[float, int]).columns.tolist()
    
    # Extract year and month from 'Perioden'
    df['Year'] = df[period_column].apply(lambda x: int(re.search(r'(\d{4})', x).group()))
    df['Month'] = df[period_column].apply(lambda x: extract_month(x))
    
    # Create a 'Quarter' column based on the month
    df['Quarter'] = df['Month'].apply(lambda x: (x - 1) // 3 + 1)
    
    # Group by 'Year' and 'Quarter' and sum the value columns
    quarterly_df = df.groupby(['Year', 'Quarter'])[value_columns].sum().reset_index()
    
    return quarterly_df

# Helper function to extract the month number from a period string
def extract_month(period_text):
    """
    Extracts the month number from the period string.
    
    Parameters:
        period_text (str): The period string to process (e.g., "2023 januari").
    
    Returns:
        int: The month number (1 for January, 2 for February, etc.).
    """
    months = {
        'januari': 1, 'februari': 2, 'maart': 3,
        'april': 4, 'mei': 5, 'juni': 6,
        'juli': 7, 'augustus': 8, 'september': 9,
        'oktober': 10, 'november': 11, 'december': 12
    }
    
    for month_name, month_num in months.items():
        if month_name in period_text.lower():
            return month_num
    
    return None  # Return None if no valid month is found

def rename_sbi_column(df, primary_sbi_column='BedrijfstakkenBranchesSBI2008', backup_sbi_column='BedrijfskenmerkenSBI2008'):
    """
    Renames the 'BedrijfskenmerkenSBI2008' column to 'BedrijfstakkenBranchesSBI2008' if the primary column does not exist.
    
    Parameters:
        df (DataFrame): The DataFrame to process.
        primary_sbi_column (str): The desired primary SBI column name. Default is 'BedrijfstakkenBranchesSBI2008'.
        backup_sbi_column (str): The backup SBI column to rename. Default is 'BedrijfskenmerkenSBI2008'.
    
    Returns:
        DataFrame: Updated DataFrame with the renamed column.
    """
    # Check if the primary column does not exist and the backup column does exist
    if primary_sbi_column not in df.columns and backup_sbi_column in df.columns:
        # Rename 'BedrijfskenmerkenSBI2008' to 'BedrijfstakkenBranchesSBI2008'
        df = df.rename(columns={backup_sbi_column: primary_sbi_column})
        print(f"Renamed '{backup_sbi_column}' to '{primary_sbi_column}'")
    
    return df

# Function to filter by industry
def filter_by_industry(df, industry_column='BedrijfstakkenBranchesSBI2008', valid_industries=None):
    """
    Filter the DataFrame by specific industries.
    
    Parameters:
        df (DataFrame): The DataFrame to filter.
        industry_column (str): Name of the industry column.
        valid_industries (list): List of valid industries to keep.
    
    Returns:
        DataFrame: Filtered DataFrame.
    """
    if valid_industries is None:
        valid_industries = [
            "Q Gezondheids- en welzijnszorg", 
            "G Handel", 
            "C Industrie", 
            # "M Specialistische zakelijke diensten", 
            # "N Verhuur en overige zakelijke diensten", 
            # "O Openbaar bestuur en overheidsdiensten"
        ]
    return df[df[industry_column].isin(valid_industries)]

# Function to filter by year range
def filter_by_year(df, year_column='Year', start_year=2006, end_year=2024):
    """
    Filter the DataFrame by a range of years.
    
    Parameters:
        df (DataFrame): The DataFrame to filter.
        year_column (str): Name of the year column.
        start_year (int): The start year (inclusive).
        end_year (int): The end year (inclusive).
    
    Returns:
        DataFrame: Filtered DataFrame.
    """
    return df[(df[year_column] >= start_year) & (df[year_column] <= end_year)]

def clean_dataframe(df, id_column='ID', year_column='Year', quarter_column='Quarter', sbi_column='BedrijfstakkenBranchesSBI2008', sick_leave_column='80072ned_Ziekteverzuimpercentage_1', columns_to_drop=None):
    """
    Cleans the DataFrame by reordering columns to have 'Year', 'Quarter', 'BedrijfstakkenBranchesSBI2008', and '80072ned_Ziekteverzuimpercentage_1' as the first four columns,
    dropping specified columns, and removing duplicate rows.
    
    Parameters:
        df (DataFrame): The DataFrame to clean.
        year_column (str): The year column name. Default is 'Year'.
        quarter_column (str): The quarter column name. Default is 'Quarter'.
        sbi_column (str): The column name for SBI. Default is 'BedrijfstakkenBranchesSBI2008'.
        sick_leave_column (str): The sick leave column name. Default is '80072ned_Ziekteverzuimpercentage_1'.
        columns_to_drop (list): List of column names to drop. Default is ['ID', 'Perioden'].
    
    Returns:
        DataFrame: Cleaned DataFrame with reordered columns, specified columns dropped, and duplicates removed.
    """
    if columns_to_drop is None:
        columns_to_drop = ['ID', 'Perioden', 'Jaar', 'Kwartaal']

    # Drop the specified columns
    df = df.drop(columns=columns_to_drop, errors='ignore')

    # Remove duplicate rows
    df = df.drop_duplicates()

    return df

def group_and_sum(df, group_by_columns):
    """
    Group the DataFrame by specified columns and sum the numeric values.
    
    Parameters:
        df (DataFrame): The DataFrame to group.
        group_by_columns (list): List of columns to group by (e.g., ['Year', 'Quarter', 'BedrijfstakkenBranchesSBI2008']).
    
    Returns:
        DataFrame: Grouped and summed DataFrame.
    """
    # Group by the specified columns and sum the numeric columns
    df = df.groupby(group_by_columns).sum(numeric_only=True).reset_index()
    
    return df

def reorder_columns(df, year_column='Year', quarter_column='Quarter', 
                    sbi_column='BedrijfstakkenBranchesSBI2008', sick_leave_column='80072ned_Ziekteverzuimpercentage_1'):
    """
    Reorders the DataFrame columns to place 'Year', 'Quarter', 'BedrijfstakkenBranchesSBI2008', 
    and '80072ned_Ziekteverzuimpercentage_1' as the first four columns, keeping remaining columns in their original order.
    
    Parameters:
        df (DataFrame): The DataFrame to reorder.
        year_column (str): The year column name.
        quarter_column (str): The quarter column name.
        sbi_column (str): The SBI column name.
        sick_leave_column (str): The sick leave column name.
    
    Returns:
        DataFrame: DataFrame with reordered columns.
    """
    # Define the preferred order of the first four columns
    preferred_columns = [year_column, quarter_column, sbi_column, sick_leave_column]

    # Collect remaining columns in their original order (excluding the preferred ones)
    remaining_columns = [col for col in df.columns if col not in preferred_columns]

    # Combine the preferred columns with the remaining ones
    ordered_columns = preferred_columns + remaining_columns

    # Reorder DataFrame
    return df[ordered_columns]


# Function to create lagged features
def create_lagged_columns(df, column, lags, group_by='BedrijfstakkenBranchesSBI2008'):
    """
    Creates lagged columns for the specified column in the DataFrame.

    Parameters:
        df (DataFrame): Input DataFrame.
        column (str): Column name to create lags for.
        lags (int): Number of lagged columns to create.
        group_by (str): Column name to group by before creating lags (default: 'BedrijfstakkenBranchesSBI2008').

    Returns:
        DataFrame: DataFrame with new lagged columns added.
    """
    df = df.copy()  # Avoid modifying the original DataFrame
    for lag in range(1, lags + 1):
        lagged_col_name = f"{column}_lag_{lag}"
        # Use .loc[] to assign the shifted values explicitly
        df[lagged_col_name] = (
            df.groupby(group_by)[column]
            .shift(lag)
            .reset_index(drop=True)  # Reset index to match df
        )
    return df


# Modify the process_tables function to handle monthly data conversion and create lagged columns
def process_tables_with_lags(tables, identifier_dict, lags=8):
    """
    Process and combine multiple tables into a single DataFrame, and create lagged columns.

    Parameters:
        tables (list): List of DataFrames.
        identifier_dict (dict): Dictionary of identifiers and their frequencies.
        lags (int): Number of lagged columns to create for '80072ned_Ziekteverzuimpercentage_1'.

    Returns:
        DataFrame: The final combined DataFrame with lagged columns.
    """
    processed_tables = []
    
    for i, table in enumerate(tables):
        identifier = list(identifier_dict.keys())[i]
        frequency = identifier_dict[identifier]

        # If the frequency is monthly, convert the data to quarterly
        if frequency == 'Maandelijks':
            print(f"Converting monthly data to quarterly for identifier: {identifier}")
            table = monthly_to_quarterly(table)

        # Step 1: Process the 'Perioden' column
        period_df = process_period_column(table)
        
        # Step 2: Fill missing 'BedrijfstakkenBranchesSBI2008' with 'BedrijfskenmerkenSBI2008'
        sbi_df = rename_sbi_column(period_df)
        
        # Step 3: Filter by industry
        industry_df = filter_by_industry(sbi_df)
        
        # Step 4: Filter by year range
        year_df = filter_by_year(industry_df)
        
        # Step 5: Clean the DataFrame: reorder columns, drop unnecessary ones, and remove duplicates
        clean_df = clean_dataframe(year_df)
        
        processed_tables.append(clean_df)
    
    # Concatenate all processed tables
    concat_df = pd.concat(processed_tables, ignore_index=True)
    
    # Step 6: Group by 'Year', 'Quarter', and 'BedrijfstakkenBranchesSBI2008' and sum the numeric columns
    grouped_df = group_and_sum(concat_df, ['Year', 'Quarter', 'BedrijfstakkenBranchesSBI2008'])
    
    # Step 7: Reorder columns for consistency
    final_df = reorder_columns(grouped_df)

    # Step 8: Create lagged columns for '80072ned_Ziekteverzuimpercentage_1'
    final_df = create_lagged_columns(final_df, '80072ned_Ziekteverzuimpercentage_1', lags)

    return final_df

In [ ]:
# Load identifiers and frequencies from the JSON file
with open('data/table_selection.json', 'r', encoding='utf-8') as json_file:
    table_data = json.load(json_file)
    identifier_dict = {entry['Identifier']: entry['Frequency'] for entry in table_data}

# Generate file paths based on identifiers
file_paths = [f'data/{identifier}.csv' for identifier in identifier_dict.keys()]

# Excluded identifiers (if applicable)
excluded_identifiers = []  # Define any identifiers you want to exclude

# Load and process data, including creating lagged columns
tables = load_csv_files(file_paths, identifier_dict, excluded_identifiers)
final_df = process_tables_with_lags(tables, identifier_dict)

# Display the first few rows of the final DataFrame
final_df.head()

## KNMI & Covid

In [ ]:
# Load KNMI data
knmi_df = pd.read_csv('data/knmi_data.csv')

# Define helper functions for adding Year_Quarter and merging KNMI data
def add_year_quarter_column(df):
    """
    Adds a 'Year_Quarter' column to the DataFrame in the format 'YYYY-QX' and removes 'Year' and 'Quarter'.
    
    Parameters:
        df (DataFrame): Input DataFrame with 'Year' and 'Quarter' columns.
    
    Returns:
        DataFrame: Modified DataFrame with 'Year_Quarter' and without 'Year' and 'Quarter' columns.
    """
    df['Year_Quarter'] = df['Year'].astype(int).astype(str) + '-Q' + df['Quarter'].astype(int).astype(str)
    return df.drop(columns=['Year', 'Quarter'])

# Define a function to label COVID-19 years
def label_covid_period(row):
    """
    Labels rows as part of the COVID-19 period based on the 'Year_Quarter' column.
    
    Parameters:
        row (Series): A row of the DataFrame.
    
    Returns:
        int: 1 if the row belongs to the COVID-19 period, 0 otherwise.
    """
    if row['Year_Quarter'].startswith('2020') or \
       row['Year_Quarter'].startswith('2021') or \
       row['Year_Quarter'] == '2022-Q1':
        return 1
    else:
        return 0

def merge_with_knmi_data(main_df, knmi_df):
    """
    Merges the main DataFrame with KNMI weather data on 'Year_Quarter' and 'BedrijfstakkenBranchesSBI2008'.
    
    Parameters:
        main_df (DataFrame): The primary DataFrame with 'Year_Quarter' and industry data.
        knmi_df (DataFrame): The KNMI data with weather details by 'Year_Quarter'.
    
    Returns:
        DataFrame: The merged DataFrame with KNMI data.
    """
    # Step 1: Create a DataFrame of unique industries from the main DataFrame
    industries = main_df[['BedrijfstakkenBranchesSBI2008']].drop_duplicates()

    # Step 2: Perform a Cartesian product to expand knmi_df across all industries
    expanded_knmi = knmi_df.merge(industries, how='cross')

    # Step 3: Merge the expanded knmi_df with the main DataFrame based on 'Year_Quarter' and industry
    merged_df = main_df.merge(expanded_knmi, on=['Year_Quarter', 'BedrijfstakkenBranchesSBI2008'], how='left')
    
    return merged_df

# Add Year_Quarter column to final_df
final_df = add_year_quarter_column(final_df)

# Apply the function to create the covid_19 column
final_df['covid_19'] = final_df.apply(label_covid_period, axis=1)

# Merge final_df with KNMI data
final_df_with_knmi = merge_with_knmi_data(final_df, knmi_df)

# Save the final DataFrame with KNMI data to a CSV file or display the head for verification
final_df_with_knmi.to_csv('data/merged_table.csv', index=False)
final_df_with_knmi.head()